<a href="https://colab.research.google.com/github/newh08/AI_Voyage99/blob/main/modelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rouge nltk evaluate rouge_score bert_score sacrebleu

import nltk
nltk.download('punkt')

In [2]:
from google.colab import files

uploaded = files.upload()


Saving ItemData.json to ItemData.json


In [3]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import random
from typing import List, Tuple
import torch


# JSON 파일 로드
with open("ItemData.json", "r", encoding="utf-8") as f:
    data = json.load(f)

random.shuffle(data)

# DataFrame 변환
df = pd.DataFrame(data)
print(df.info())

# 데이터 분할 (90% train, 10% validation)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

schema = '''Table : "customers" Column: "customer_id", "first_name", "last_name", "email", "phone_number", "address_id",
    foreign_key: "address_id" from "addresses", "address_id",
    foreign_key: "customer_id" from "orders", "customer_id" [SEP]
Table : "addresses" Column: "address_id", "street", "city", "state", "zip_code", "country_id",
    foreign_key: "country_id" from "countries", "country_id" [SEP]
Table : "orders" Column: "order_id", "customer_id", "order_date", "total_amount", "status",
    foreign_key: "customer_id" from "customers", "customer_id",
    foreign_key: "order_id" from "order_items", "order_id" [SEP]
Table : "order_items" Column: "order_item_id", "order_id", "product_id", "quantity", "price",
    foreign_key: "order_id" from "orders", "order_id",
    foreign_key: "product_id" from "products", "product_id" [SEP]
'''

def create_prompt(question: str) -> str:
    return f"Generate an SQL query using the schema below.:\nSchema: {schema}\nQuestion: {question}"

def create_vals_and_labels(df: pd.DataFrame):
    list_prompts = df["question"].apply(create_prompt).tolist()
    list_labels = df["label"].tolist()
    return list_prompts, list_labels

# 학습 데이터셋
prompts_train, labels_train = create_vals_and_labels(train_df)

# 검증 데이터셋
prompts_val, labels_val = create_vals_and_labels(val_df)

print(f"📌 Train 데이터 개수: {len(prompts_train)}")
print(f"📌 Validation 데이터 개수: {len(prompts_val)}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  440 non-null    object
 1   label     440 non-null    object
dtypes: object(2)
memory usage: 7.0+ KB
None
📌 Train 데이터 개수: 396
📌 Validation 데이터 개수: 44


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# 모델 로드
model_path = "gaussalgo/T5-LM-Large-text2sql-spider"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

model.to(device)

# 🔹 모델을 사용하여 SQL 예측
def generate_sql(model, tokenizer, questions):
    sql_outputs = []
    for question in questions:
        inputs = tokenizer(question, return_tensors="pt").to(device)  # 입력 토큰화
        outputs = model.generate(**inputs, max_length=512)  # SQL 생성
        sql = tokenizer.decode(outputs[0], skip_special_tokens=True)  # 디코딩
        sql_outputs.append(sql)
    return sql_outputs

# 검증 데이터 SQL 생성
predicted_sqls = generate_sql(model, tokenizer, prompts_val)

# 결과 샘플 확인
for i in range(3):
    print(f"🔹 질문: {prompts_val[i]}")
    print(f"🔹 실제 SQL: {labels_val[i]}")
    print(f"🔹 모델 예측 SQL: {predicted_sqls[i]}\n")

from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(predicted_sqls, labels_val, avg=True)

print(f"✅ 학습 전 ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"✅ 학습 전 ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"✅ 학습 전 ROUGE-L: {scores['rouge-l']['f']:.4f}")

from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# BLEU 점수 계산
smoothie = SmoothingFunction().method1
bleu_score = corpus_bleu([[label.split()] for label in labels_val],
                         [pred.split() for pred in predicted_sqls],
                         smoothing_function=smoothie)

print(f"✅ 학습 전 BLEU Score: {bleu_score:.4f}")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Using device: cuda
🔹 질문: Generate an SQL query using the schema below.:
Schema: Table : "customers" Column: "customer_id", "first_name", "last_name", "email", "phone_number", "address_id",
    foreign_key: "address_id" from "addresses", "address_id",
    foreign_key: "customer_id" from "orders", "customer_id" [SEP]
Table : "addresses" Column: "address_id", "street", "city", "state", "zip_code", "country_id",
    foreign_key: "country_id" from "countries", "country_id" [SEP]
Table : "orders" Column: "order_id", "customer_id", "order_date", "total_amount", "status",
    foreign_key: "customer_id" from "customers", "customer_id",
    foreign_key: "order_id" from "order_items", "order_id" [SEP]
Table : "order_items" Column: "order_item_id", "order_id", "product_id", "quantity", "price",
    foreign_key: "order_id" from "orders", "order_id",
    foreign_key: "product_id" from "products", "product_id" [SEP]

Question: List addresses along with country_id.
🔹 실제 SQL: SELECT address_id, street,

In [5]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
import evaluate
import numpy as np
from datasets import Dataset
import pandas as pd

# ✅ Hugging Face Dataset 변환
train_dataset = Dataset.from_pandas(pd.DataFrame({"text": prompts_train, "labels": labels_train}))
val_dataset = Dataset.from_pandas(pd.DataFrame({"text": prompts_val, "labels": labels_val}))

# 데이터셋 토크나이징
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(examples["labels"], padding="max_length", truncation=True, return_tensors="pt")
    model_inputs["labels"] = labels["input_ids"]

    model_inputs = {key: val.to(device) for key, val in model_inputs.items()}
    return model_inputs

# 데이터셋 토크나이징 및 MPS 이동
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

tokenized_train.set_format("torch")
tokenized_val.set_format("torch")


# BLEU 및 ROUGE 메트릭 로드
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def compute_metrics(pred):
    predictions, labels = pred

    # 확률값(logits)이라면 argmax 적용
    if isinstance(predictions, tuple):
        predictions = predictions[0]  # logits 선택
    predictions = np.argmax(predictions, axis=-1)  # 가장 높은 확률의 토큰 선택

    # NumPy 배열 또는 Tensor를 리스트로 변환
    predictions = predictions.tolist() if isinstance(predictions, np.ndarray) else predictions
    labels = labels.tolist() if isinstance(labels, np.ndarray) else labels

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU 계산
    bleu_score = bleu_metric.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    # ROUGE 계산
    rouge_score = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "bleu": bleu_score["bleu"],
        "rouge1": rouge_score["rouge1"],
        "rouge2": rouge_score["rouge2"],
        "rougeL": rouge_score["rougeL"],
    }



training_args = TrainingArguments(
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # epoch 수
    per_device_train_batch_size=4,  # training data의 batch size
    per_device_eval_batch_size=4,  # validation data의 batch size
    logging_strategy="steps",  # steps 끝날 때마다 training loss 등을 log하라는 의미
    logging_steps=10,  # 매 10 스텝마다 로깅
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    evaluation_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-5,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

accuracy = evaluate.load("accuracy")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

Map:   0%|          | 0/396 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<ipython-input-5-81d1a232f514>:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: newh08 (newh008) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel
1,15.498400,8.269021,0.161659,0.376717,0.142830,0.349961
2,0.297200,0.112855,0.418163,0.597690,0.392323,0.579723
3,0.093800,0.066440,0.570594,0.780142,0.575859,0.762141
4,0.090200,0.054717,0.622280,0.805425,0.621679,0.794984
5,0.060100,0.045428,0.611423,0.739532,0.570414,0.726113
6,0.070100,0.037354,0.652094,0.826426,0.647242,0.814219
7,0.075000,0.032147,0.680985,0.843244,0.678823,0.830374
8,0.052800,0.030466,0.688364,0.854035,0.692044,0.841643
9,0.045400,0.029630,0.708717,0.857238,0.699995,0.847923
10,0.050900,0.029328,0.710690,0.860030,0.708595,0.849715


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=990, training_loss=2.6564847266433214, metrics={'train_runtime': 1211.881, 'train_samples_per_second': 3.268, 'train_steps_per_second': 0.817, 'total_flos': 9126881085358080.0, 'train_loss': 2.6564847266433214, 'epoch': 10.0})

In [6]:
# 학습 후 재평가
predicted_sqls_after_train = generate_sql(model, tokenizer, prompts_val)

scores = rouge.get_scores(predicted_sqls_after_train, labels_val, avg=True)

print(f"✅ 학습 후 ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"✅ 학습 후 ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"✅ 학습 후 ROUGE-L: {scores['rouge-l']['f']:.4f}")


bleu_score = corpus_bleu([[label.split()] for label in labels_val],
                         [pred.split() for pred in predicted_sqls_after_train],
                         smoothing_function=smoothie)

print(f"✅ 학습 후 BLEU Score: {bleu_score:.4f}")


✅ 학습 후 ROUGE-1: 0.7020
✅ 학습 후 ROUGE-2: 0.5192
✅ 학습 후 ROUGE-L: 0.6976
✅ 학습 후 BLEU Score: 0.3974
